# Instructions for Reproduction

1. Run Figures_1,2,3.ipynb;
2. Run this notebook;
3. Run reproduce_all.sh script;

## Preparation

#### Directories

In [ ]:
%%bash

mkdir ../results
mkdir ../unpacked
mkdir ../images

#### Images

In [20]:
import os
import cv2
from misc import *
import pandas as pd
import os.path as op
from tqdm import tqdm

In [21]:
ru = Study.load_from_file("ru_study.pkl").compute_word_set()
en = Study.load_from_file("en_study.pkl").compute_word_set()

In [23]:
for k in [ru, en]:
    for a in tqdm(k):
        u = Study.onehot(a, color=True)
        u = u.reshape(int(u.shape[0]/13), 13, 3)
        cv2.imwrite(op.join("../images/", a+".png"), u)

100%|██████████| 241/241 [00:00<00:00, 2743.41it/s]


## Dataset construction

In [1]:
EN_VEC = "/home/bakirillov/HDD/weights/fasttext/aligned/wiki.en.align.vec"
RU_VEC = "/home/bakirillov/HDD/weights/fasttext/aligned/wiki.ru.align.vec"
VC = "python vectors.py -v VEC -s STUDY -w WHAT -o OUTPUT"

In [2]:
script = open("unpack_data_PI.sh", "w")
script.write("#!/bin/sh\n\n\n")
for a in ["wv", "1hot"]:
    for b,c in zip([EN_VEC, RU_VEC], ["en_study.pkl", "ru_study.pkl"]):
        command = VC.replace("WHAT", a).replace("VEC", b).replace(
            "STUDY", c
        ).replace("OUTPUT", "../unpacked/"+c+"_"+a+".csv")+"\n"
        script.write(command)
script.close()

In [3]:
VC = "python vectors.py -v VEC -s STUDY -w WHAT -p PARTICIPANT -o OUTPUT"
script = open("unpack_data_PD.sh", "w")
script.write("#!/bin/sh\n\n\n")
for a in ["wv", "1hot"]:
    for b,c,d in zip(
        ["../unpacked/en_study.pkl_wv.csv", "../unpacked/ru_study.pkl_wv.csv"], 
        ["en_study.pkl", "ru_study.pkl"], [34, 103]
    ):
        for e in range(d):
            command = VC.replace("WHAT", a).replace("VEC", b).replace(
                "STUDY", c
            ).replace(
                "OUTPUT", "../unpacked/"+c+"_"+a+"_"+str(e)+".csv"
            ).replace("PARTICIPANT", str(e))+"\n"
            script.write(command)
script.close()

In [4]:
%%bash

sh unpack_data_PI.sh
sh unpack_data_PD.sh

2519370it [00:57, 43775.98it/s]
1888423it [00:49, 38077.70it/s]


## Model training

In [6]:
files = [op.join("../unpacked/", b) for b in [a for a in os.walk("../unpacked/")][0][2]]

### Participant-Independent word vector

In [7]:
VC = "python PI.py -d DATASET -a 3 -s 10 -o OUTPUT"

In [8]:
current = list(filter(lambda x: "_wv.csv" in x, files))
pd.concat([pd.read_csv(a) for a in current]).to_csv("../unpacked/joint_wv.csv")
current.append("../unpacked/joint_wv.csv")

In [9]:
script = open("reproduce_PI_wv.sh", "w")
script.write("#!/bin/sh\n\n\n")
for a in current:
    command = VC.replace(
        "DATASET", a
    ).replace(
        "OUTPUT", a.replace("unpacked", "results").replace(".csv", "")
    )
    script.write(command+"\n")
script.close()

### Participant-Independent one-hot

In [16]:
VC = "python PI.py -d DATASET -a 0 -s 10 -o OUTPUT"

In [17]:
current = list(filter(lambda x: "_1hot.csv" in x, files))

In [18]:
script = open("reproduce_PI_1hot.sh", "w")
script.write("#!/bin/sh\n\n\n")
for a in current:
    command = VC.replace(
        "DATASET", a
    ).replace(
        "OUTPUT", a.replace("unpacked", "results").replace(".csv", "")
    )
    script.write(command+"\n")
script.close()

### Participant-Dependent word vector

In [25]:
current = list(filter(lambda x: "_wv_" in x, files))

In [26]:
VC = "python PD.py -d DATASET -a 3 -s 10 -o OUTPUT"

In [27]:
script = open("reproduce_PD_wv.sh", "w")
script.write("#!/bin/sh\n\n\n")
for a in current:
    command = VC.replace(
        "DATASET", a
    ).replace(
        "OUTPUT", a.replace("unpacked", "results").replace(".csv", "")
    )
    script.write(command+"\n")
script.close()

### Participant-Dependent 1hot

In [28]:
current = list(filter(lambda x: "_1hot_" in x, files))

In [29]:
VC = "python PD.py -d DATASET -a 0 -s 10 -o OUTPUT"

In [30]:
script = open("reproduce_PD_1hot.sh", "w")
script.write("#!/bin/sh\n\n\n")
for a in current:
    command = VC.replace(
        "DATASET", a
    ).replace(
        "OUTPUT", a.replace("unpacked", "results").replace(".csv", "")
    )
    script.write(command+"\n")
script.close()